# Multilayer Perceptron (MLP) for generating Onion-like News Headlines

Based on Andrej Karpathy's Youtube lecture [Building makemore Part 2: MLP](https://www.youtube.com/watch?v=TCH_1BHY58I) and Bengio et al's paper [A Neural Probabilistic Langue Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, sys, warnings, os, json, torch, re
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML
from pathlib import Path

import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

from torch.nn import functional as F
import torch.nn as nn

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

In [2]:
onion_df = pd.read_csv('../data/cleaned_onion_headlines.csv')

## Bigram Language Model

In [ ]:
texts = onion_df['text'].tolist()
vocab = ['#'] + sorted(list(set(' '.join(texts))))
stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for i,s in enumerate(vocab)}
len(vocab)

In [ ]:
xs,ys = [],[]
for text in texts[:1]:
  chs = ['#'] + list(text) + ['#']
  for ch1, ch2 in zip(chs, chs[1:]):
    idx1 = stoi[ch1]
    idx2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(idx1)
    ys.append(idx2)

xs,ys = torch.tensor(xs),torch.tensor(ys)

In [ ]:
xs

In [ ]:
ys

Make sure to cast the encoding to `float` because we don't want to pass `int` into the neural network

In [ ]:
xenc = F.one_hot(xs, num_classes=55).float()
plt.imshow(xenc)

We interpret that the NN outputs `logcounts`

In [ ]:
xenc.shape

In [ ]:
g = torch.Generator().manual_seed(2468975301)
W = torch.randn((len(vocab), len(vocab)), generator=g)

Lines 2-3 is basically `softmax`

In [ ]:
xenc = F.one_hot(xs, num_classes=len(vocab)).float() # input to the network: one-hot encoding
logits = (xenc @ W)
counts = logits.exp() # equivalent to bigram_counts
probs = counts/counts.sum(axis=1, keepdims=True)
probs

In [ ]:
nlls = torch.zeros(len(xenc))
for i in range(len(xenc)):
  x = xs[i].item() # input character idx
  y = ys[i].item() # label character idx  
  print(f"bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x}, {y})")
  print(f"input to the NN: {x}")
  print(f"output probabilities from NN: {probs[i]}")
  print(f"label (actual next character): {y}")
  p = probs[i, y]
  print(f"probability assigned by the NN to the correct character: {p.item()}")
  logp = torch.log(p)
  print(f"log liklihood: {logp.item()}")
  nll = -logp
  print(f"negative log liklihood: {nll.item()}")
  nlls[i] = nll
  print("-"*50)

print("="*50)  
print(f"average nll: {nlls.mean().item()}")

In [ ]:
xs

In [ ]:
ys

In [ ]:
g = torch.Generator().manual_seed(2468975301)
W = torch.randn((len(vocab), len(vocab)), generator=g, requires_grad=True)

Pluck out the probs corresponding to the indices in `ys`

This is the forward pass

In [ ]:
# xenc = F.one_hot(xs, num_classes=len(vocab)).float() # input to the network: one-hot encoding
# logits = (xenc @ W)
logits = W[xs]

# counts = logits.exp() # equivalent to bigram_counts
# probs = counts/counts.sum(axis=1, keepdims=True)
# loss = -probs[torch.arange(len(ys)), ys].log().mean()

loss = F.cross_entropy(logits, ys)
print(f"Loss: {loss.item():0.4f}")

In [ ]:
# Backward pass
W.grad = None # set grad to zero
loss.backward()
W.data += -0.1 * W.grad

## Putting it all together

In [ ]:
on_gpu = True
device = 'cuda' if on_gpu else 'cpu'

In [ ]:
# create the dataset
xs,ys = [],[]
for text in texts:
  chs = ['#'] + list(text) + ['#']
  for ch1, ch2 in zip(chs, chs[1:]):
    idx1 = stoi[ch1]
    idx2 = stoi[ch2]
    xs.append(idx1)
    ys.append(idx2)

xs,ys = torch.tensor(xs, device=device),torch.tensor(ys, device=device)
num = xs.nelement()
print(f"Number of examples: {num}")

# initialize the NN
g = torch.Generator(device=device).manual_seed(2468975301)
W = torch.randn((len(vocab), len(vocab)), generator=g, requires_grad=True, device=device)

In [ ]:
%%time
n_epochs = 250
# gradient descent
for _ in range(n_epochs):
  # forward pass
  logits = W[xs]
  loss = F.cross_entropy(logits, ys)
  # loss = -F.cross_entropy(logits, ys) + 0.01 * (W**2).mean()
  
  # backward pass
  W.grad = None # set grad to zero
  loss.backward()
  # print(f"Loss: {loss.item():0.4f}")  
  
  # update
  W.data += -50 * W.grad
  
print(f"Final Loss: {loss.item():0.4f}")

In [ ]:
W = W.cpu()

In [ ]:
g = torch.Generator().manual_seed(2468975301)
n_headlines = 20
for _ in range(n_headlines):
  idx = 0
  ai_onion = []
  while True:    
    # xenc = F.one_hot(torch.tensor([idx]), num_classes=len(vocab)).float()
    # logits = xenc @ W
    logits = W[idx].unsqueeze(0)
    p = logits.exp() / logits.exp().sum(1, keepdims=True)
    idx = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    # idx = torch.multinomial(torch.ones(len(vocab))/len(vocab), num_samples=1, replacement=True, generator=g).item()
    ai_onion.append(itos[idx])
    if idx == 0:
      break
  print(''.join(ai_onion[:-1]))    